In [1]:
from utils.imports import *

Using TensorFlow backend.


In [2]:
def get_dirfiles(dir):
    file_list = []
    subset_path = os.listdir(dir)
    for _ in range(len(subset_path)):
        if subset_path[_] != '.DS_Store':
            file_list.append(dir + subset_path[_])
    return file_list

def train_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[0:19]
    file_list_false = get_dirfiles(output_false)[0:19]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)
    
    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,10,30,30])
    labels = np.zeros([nb_true,2])
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def valid_generator(output_true,output_false):
    file_list_true = get_dirfiles(output_true)[-3:]
    file_list_false = get_dirfiles(output_false)[-3:]
    
    file_list_true = shuffle(file_list_true)
    file_list_false = shuffle(file_list_false)

    nb_true = len(file_list_true) + len(file_list_false)
    sample = np.zeros([nb_true,10,30,30])
    labels = np.zeros([nb_true,2])
  
    for i in tqdm(range(len(file_list_true))):       
        cc= np.load(file_list_true[i])
        sample[i] = cc
        labels[i] = [0.,1.]
    for j in tqdm(range(len(file_list_false))):
        bb= np.load(file_list_false[j])
        sample[j+len(file_list_true)] = bb 
        labels[j+len(file_list_true)] = [1.,0.]
    sample = np.expand_dims(sample, axis=1)        
    return sample,labels

def model_30():    

    inputs = Input(shape=(1, 10, 30, 30))
    
    conv1 = Convolution3D(48, 3, 5, 5, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(48, 1, 2, 2, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(48, 3, 5, 5, activation = 'relu', border_mode='same')(inputs)
    conv1 = BatchNormalization(axis = 1)(conv1)
    conv1 = Convolution3D(48, 3, 5, 5, activation = 'relu', border_mode='same')(conv1)
    conv1 = BatchNormalization(axis = 1)(conv1)    
    
    output = Flatten(name='flatten')(conv1)
    output = Dense(250)(output)
    output = PReLU()(output)
    output = BatchNormalization()(output)
    output = Dense(2, activation='softmax', name = 'predictions')(output)
    model3d = Model(inputs, output)
    model3d.compile(loss='categorical_crossentropy', optimizer = Adam(lr=1e-5), metrics = ['accuracy'])
    return model3d


def fenlei_fit(name, load_check = False,batch_size=2, epochs=100,check_name = None):

    t = time.time()
    callbacks = [EarlyStopping(monitor='val_loss', patience = 20, verbose = 1),
                 ModelCheckpoint((model_paths + '{}.h5').format(name),
                                 monitor='val_loss',
                                 verbose = 0,
                                 save_best_only = True)]
    if load_check:
        check_model = (model_paths + '{}.h5').format(check_name)
        model = load_model(check_model)
    else:
        model = model_30()
    x,y = train_generator(output_true,output_false)
    model.fit(x=x, y=y, batch_size=batch_size, epochs=epochs,
              validation_data=valid_generator(output_true,output_false),verbose=1, callbacks=callbacks, shuffle=True) 
    return model

In [ ]:
output_path = PATH['cls_train_30']
output_true = PATH['cls_train_30_true']
output_false = PATH['cls_train_30_false']
model_paths = PATH['model_paths']
model_final = PATH['model_final']

In [ ]:
fenlei_fit('Fenge_10_30_30_0617_2', load_check = False, batch_size=1, epochs=100, check_name = 'Fenge_10_30_30_0617')

100%|██████████| 3/3 [00:00<00:00, 208.48it/s]


Train on 38 samples, validate on 6 samples
Epoch 1/100
38/38 [==============================] - 24s - loss: 0.6932 - acc: 0.4737 - val_loss: 0.7311 - val_acc: 0.5000
Epoch 2/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.5000 - val_loss: 0.8673 - val_acc: 0.5000
Epoch 3/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.5000 - val_loss: 1.0795 - val_acc: 0.5000
Epoch 4/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.3684 - val_loss: 1.3133 - val_acc: 0.5000
Epoch 5/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.5000 - val_loss: 1.5529 - val_acc: 0.5000
Epoch 6/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.4737 - val_loss: 1.7569 - val_acc: 0.5000
Epoch 7/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.5000 - val_loss: 1.9484 - val_acc: 0.5000
Epoch 8/100
38/38 [==============================] - 5s - loss: 0.6932 - acc: 0.4211 - val_loss: 2.053

In [ ]:
file_list_true = get_dirfiles(output_true)
file_list_false = get_dirfiles(output_false)

In [ ]:
#model_pred = classifier((1, 36, 36, 36), (3, 3, 3), (2, 2, 2))
model_pred = load_model(model_paths + 'Fenge_10_30_30_0617_2.h5')

In [ ]:
cc = []
for i in file_list_false[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0
for i in cc:
    if i[0][0] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc

In [ ]:
cc = []
for i in file_list_true[0:200]:
    a=np.load(i)
    a=np.expand_dims(a,0)
    a=np.expand_dims(a,0)
    cc.append(model_pred.predict(a))
count = 0


In [ ]:
for i in cc:
    if i[0][1] > 0.9:
        count += 1
print count*1.0/len(cc)

In [ ]:
cc